# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
cities_list_df = pd.read_csv('../WeatherPy/City_List.csv')
cities_list_df.drop(columns=['Unnamed: 0'], inplace=True)
cities_list_df

,City,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Date
0,andorra la vella,42.50,1.49,37.00,74,0,5.37,AD,1602989388
1,agarakadzor,39.73,45.35,42.80,70,0,4.70,AM,1602989389
2,berd,40.88,45.39,53.94,61,0,3.89,AM,1602989389
3,lanjaghbyur,40.27,45.14,44.60,81,0,4.23,AM,1602989389
4,shahumyan,39.94,44.57,44.60,81,0,0.98,AM,1602989389
...,...,...,...,...,...,...,...,...,...
611,tam ky,15.57,108.48,84.20,83,75,1.12,VN,1602989526
612,ekangala,-25.70,28.75,69.01,82,0,4.36,ZA,1602989526
613,phalaborwa,-23.94,31.14,69.80,73,0,4.70,ZA,1602989526
614,willowmore,-33.29,23.49,58.06,63,61,2.28,ZA,1602989527


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
locations = cities_list_df[["Lat", "Lng"]].astype(float)
humidity = cities_list_df['Humidity(%)'].astype(float)

fig = gmaps.figure()

# adds heatmap layet to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
ideal_cities_df = cities_list_df.loc[(cities_list_df['Max Temp(F)'] > 70) & (cities_list_df['Max Temp(F)'] < 80)]
ideal_cities_df = ideal_cities_df.loc[ideal_cities_df['Humidity(%)'] < 55]
ideal_cities_df = ideal_cities_df.loc[ideal_cities_df['Cloudiness(%)'] == 0]
ideal_cities_df

,City,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Date
197,san buenaventura,27.08,-101.53,78.01,43,0,3.00,MX,1602989432
232,hastinapur,29.17,78.02,75.99,35,0,3.38,IN,1602989440
241,makhu,31.10,75.00,77.41,19,0,1.25,IN,1602989442
250,ramkola,26.90,83.84,79.07,52,0,4.03,IN,1602989444
254,sinnar,13.57,33.57,79.03,50,0,7.43,SD,1602989445
553,coalinga,36.14,-120.36,78.01,40,0,1.95,US,1602989514
555,madera,37.17,-119.83,73.99,36,0,5.46,US,1602989462
615,siavonga,-16.54,28.71,74.79,45,0,2.53,ZM,1602989527


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = ideal_cities_df
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Date,Hotel Name
197,san buenaventura,27.08,-101.53,78.01,43,0,3.00,MX,1602989432,
232,hastinapur,29.17,78.02,75.99,35,0,3.38,IN,1602989440,
241,makhu,31.10,75.00,77.41,19,0,1.25,IN,1602989442,
250,ramkola,26.90,83.84,79.07,52,0,4.03,IN,1602989444,
254,sinnar,13.57,33.57,79.03,50,0,7.43,SD,1602989445,
553,coalinga,36.14,-120.36,78.01,40,0,1.95,US,1602989514,
555,madera,37.17,-119.83,73.99,36,0,5.46,US,1602989462,
615,siavonga,-16.54,28.71,74.79,45,0,2.53,ZM,1602989527,


In [70]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    city = row['City']
    
    base_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{long}&radius=5000&type=lodging&key={g_key}'
    
    hotel_info = requests.get(base_url).json()
    
    # returns hotel info near each city but doesn't break out of loop if there's an IndexError due to no city in API
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_info['results'][0]['name']
        print(f'We found a hotel in {city}!')
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = 'City not in the Google API!'
        print(f'{city} is not in the Google API!')

We found a hotel in san buenaventura!
We found a hotel in hastinapur!
We found a hotel in makhu!
We found a hotel in ramkola!
We found a hotel in sinnar!
We found a hotel in coalinga!
madera is not in the Google API!
We found a hotel in siavonga!


In [71]:
# completed df
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Date,Hotel Name
197,san buenaventura,27.08,-101.53,78.01,43,0,3.00,MX,1602989432,Hotel Gran Plaza
232,hastinapur,29.17,78.02,75.99,35,0,3.38,IN,1602989440,SHRADDHA SADAN (श्रद्धा सदन)
241,makhu,31.10,75.00,77.41,19,0,1.25,IN,1602989442,Guru Nanak Tent House
250,ramkola,26.90,83.84,79.07,52,0,4.03,IN,1602989444,Pankaj Sahani Ramkola
254,sinnar,13.57,33.57,79.03,50,0,7.43,SD,1602989445,فندق الرشيد
553,coalinga,36.14,-120.36,78.01,40,0,1.95,US,1602989514,Royal Lodge Motel
555,madera,37.17,-119.83,73.99,36,0,5.46,US,1602989462,City not in the Google API!
615,siavonga,-16.54,28.71,74.79,45,0,2.53,ZM,1602989527,Lavongora Waterfront Lodge


In [84]:
locations = hotel_df[["Lat", "Lng"]]
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=10,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))